Alright onto more helio-relevant problems.  Let's consider diffusive inertial waves in a differentially rotating system with a constant density.  This system can be written as

$$
\partial_t \vec{u} + \nabla p + \hat{e}_z\times \vec{u} + \left(\nabla\times\vec{u}\right)\times\vec{v}_0 + \vec{\omega}_0\times\vec{u} - Ek \nabla^2 \vec{u} = 0,\\
\nabla\cdot\vec{u} = 0,
$$

where $Ek$ is the Ekman number defined here as $\nu/(2\Omega_0 R^2)$, with $\nu$ the kinematic viscosity, $\Omega_0$ the bulk rotation rate, and $R$ the radius of the outer bounding sphere. Likewise, $\vec{u}$ is the velocity, and $p$ a gauge field. Turning this into an eigenproblem, we assume oscillatory modes such that $\vec{u} = \overline{u} e^{-i\omega t}$, more generally there can be non-normal behavior such that $\vec{u} = \overline{u} g(t)$ for some hopefully bounded $g(t)$. To assess that we can use pseudospectra, but I'll save that for another time.

For now let us also consider all background velocities $\vec{v}_0$ and vorticities $\vec{\omega}_0$ in the rotating frame to be zero.

In [ ]:
#Let's get ourselves started with a few module imports.
import os
import numpy as np
import dedalus.public as d3
from dedalus.core import evaluator #currently necessary, will be obsolete soon.

In [ ]:
#Datatype
dtype = np.complex128

#Problem Parameters
nphi=1 #We are going to look at axisymmetric waves here.
ntheta=128 #Number of grid points in latitude (Legendre weights)
nr=32 #Number of grid points in radius (Chebyshev weights)

ri = 0.70    #Inner radius
ro = 1.00    #Outer radius
Ek = 1e-4    #Ekman number

# Set up output directory for eigenmodes
label='0'
data_dir = 'Inertial_Waves_Ek{:.2f}'.format(np.log10(Ek))+'_'+label

#Define radii for coordinate system creators
radii = (ri,ro)

In [ ]:
#For eigenproblems there's no MPI mesh for dedalus, but there can be for SLEPc.
mesh = None
# Bases
coords = d3.SphericalCoordinates('phi', 'theta', 'r') #Define the spherical coordinate names
dist = d3.Distributor(coords, mesh=mesh, dtype=dtype) #Set up the internal communicators and initialize.
#Define a shell basis (so Tensor spherical harmonics and a Chebyshev radial basis)
shell = d3.ShellBasis(coords, shape=(nphi,ntheta,nr), radii=radii, dtype=dtype)
sphere = shell.outer_surface #Define the bounding sphere coordinate systems (e.g. S_2)
phi, theta, r = dist.local_grids(shell) #Get local coordinate arrays

In [ ]:
# Fields over the shell
u = dist.VectorField(coords, bases=shell, name='u') #Vector velocity field
p = dist.Field(bases=shell, name='p') #Gauge field.

#Vertical unit vector in spherical coordinates
#This guy is special: it is defined over a meridional basis 
# (e.g. coupled in radius and theta, but not in phi).
#This restricted basis is called the meridional_basis.
ez = dist.VectorField(coords, bases=shell.meridional_basis, name='ez')

# Eigenvalue
omega = dist.Field(name='omega')

In [ ]:
#Tau boundary mullifiers (dummy variables permitting the
# our spectral representation of boundary conditions)
tau_u_ri = dist.VectorField(coords, bases=sphere, name='tau_u_ri')
tau_u_ro = dist.VectorField(coords,bases=sphere, name='tau_u_ro')
tau_p = dist.Field(name='tau_p')

In [ ]:
#Substitutions

dt = lambda A: -1j*omega*A #Our eigenvalue operator.

#Here we lift the taus onto the shell derivative basis 
#(So ChebyU in radius and the shifted Jacobi polynomials in latitude)
lift_basis = shell.derivative_basis(1) # First derivative basis
lift = lambda A: d3.Lift(A, lift_basis, -1)

In [ ]:
#First order formulation
rvec = dist.VectorField(coords, bases=shell.meridional_basis, name='rvec')
#Radial component, since chebyshev in radius
rvec['g'][2] = r
#Define the gradient tensor and the first lift for the lower boundary
grad_u = d3.grad(u) + rvec*lift(tau_u_ri)

#For stress-free boundary conditions
strain_rate = d3.grad(u) + d3.transpose(d3.grad(u))

#Vertical unit vector
ez = dist.VectorField(coords, bases=shell.meridional_basis, name='ez')
#Theta component
ez['g'][1] = -np.sin(theta)
#Radial component
ez['g'][2] = np.cos(theta)

In [ ]:
# Problem
problem = d3.EVP([p, u, tau_u_ri, tau_u_ro, tau_p], eigenvalue=omega, namespace=locals())

#Equations of motion
#Continuity + a tau to control the gauge field.
problem.add_equation("trace(grad_u) + tau_p = 0")
#Momentum equation with the Coriolis force.
problem.add_equation("dt(u) + grad(p) + cross(ez,u) - Ek*div(grad_u) + lift(tau_u_ro) = 0")

In [ ]:
# Pressure Gauge
problem.add_equation("integ(p) = 0")

#Boundary conditions
stress_free=False
if (stress_free):
    problem.add_equation("radial(u(r=ri)) = 0") #Impenetrable lower boundary
    problem.add_equation("angular(radial(strain_rate(r=ri),0),0) = 0") #Stress-free lower boundary
    problem.add_equation("radial(u(r=ro)) = 0") #Impenetrable upper boundary
    problem.add_equation("angular(radial(strain_rate(r=ro),0),0) = 0") #Stress-free upper boundary
else:
    problem.add_equation("u(r=ri)=0") #No-slip boundaries
    problem.add_equation("u(r=ro)=0")

In [ ]:
solver = problem.build_solver()
ss = solver.subproblems[0].subsystems[0]

#Let's use the sparse solver (waaaaay faster).
dense=False

#Let's ask for 128 eigenvalues
neig = 128

#Let's set a target for low frequency modes
guess = 0e0

if (dense):
    solver.solve_dense(solver.subproblems[0])
else:
    solver.solve_sparse(solver.subproblems[0], neig, guess)


In [ ]:
namespace = {}
solver.evaluator = evaluator.Evaluator(solver.dist, namespace)

if not os.path.exists('{:s}'.format(data_dir)):
    os.mkdir('{:s}'.format(data_dir))

np.save(data_dir+'/eigenvalues.npy',solver.eigenvalues)
path = data_dir + '/checkpoints'
checkpoint = solver.evaluator.add_file_handler(path, max_writes=1)
checkpoint.add_tasks(solver.state)

for i in range(neig):
    solver.set_state(i, ss)
    solver.evaluator.evaluate_handlers([checkpoint],sim_time=1,wall_time=1,timestep=1,iteration=1)

In [ ]:
import matplotlib.pyplot as plt

fig,ax=plt.subplots()

exists  = os.path.isdir(data_dir)
if (exists):
    eigs = np.load(data_dir+'/eigenvalues.npy')
    ax.scatter(eigs.real,eigs.imag,label='128x32',marker='x',alpha=1,s=50)

ax.set_xlabel('Real')
ax.set_ylabel('Imag')
ax.legend()
fig.savefig('Eigs_compare_0.png', dpi=600)

Let's see what some of these modes look like!

In [ ]:
#Find a mode:
idx = np.where(eigs.imag>-0.1)[0]
phig, thetag, rg = shell.global_grids((1, 4, 3/2)) #Get local coordinate arrays
thetas = np.array(thetag).flatten()
rads = np.array(rg).flatten()

for jj in idx:
    solver.set_state(jj, ss)
    u.change_scales((1,4,3/2))
    u.require_grid_space()

    p.change_scales((1,4,3/2))
    p.require_grid_space()

    x = thetas
    y = rads
    x = np.pi/2 - x
    xx, yy = np.meshgrid(x, y)

    fig = plt.figure(figsize=[8,6],dpi=200,constrained_layout=False,linewidth=2.0)
    data = [u['g'][2][0],u['g'][1][0],u['g'][0][0],p['g'][0]]
    labels = ['(a)','(b)','(c)','(d)']
    labels2 = [r'$u_r$',r'$u_\theta$',r'$u_\phi$',r'$p$']
    cmaps = ['PiYG','PRGn','RdYlGn','RdYlBu']
    saturation = 0.99

    for ii in np.arange(4):
        pos = [-0.21+ii*0.16,0.02,0.9,0.96]
        pb = fig.add_axes(pos,projection='polar')
        pb.set_xticklabels([])
        pb.set_yticklabels([])
    
        field = data[ii].real
        print(np.shape(field))
        vals = np.sort(np.abs(field.flatten()))
        vals = np.sort(vals)
        vmax = vals[int(saturation*len(vals))]
        vmin = -vmax
        print(vmin,vmax)
   
        meplot = pb.pcolormesh(xx, yy, field.T, cmap=cmaps[ii], vmin=vmin, vmax=vmax, rasterized=False, shading='auto')
        #if (ii % 2 ==0):
        #    meplot2 = pb.contour(xx,yy,field.T,16,colors='xkcd:grey')
            
        pb.set_theta_offset(0)
        pb.set_thetalim([-np.pi/2,np.pi/2])
        pb.set_rlim(0.89,0.99)
        pb.set_rorigin(0)
        pb.set_aspect(1)
        pb.grid(False)
        pb.text(0.33,0.95,labels[ii],transform=pb.transAxes,fontsize=20)
        pb.text(0.62,0.03,labels2[ii],transform=pb.transAxes,fontsize=20)
    
    fig.savefig('Inertial_Wave_Eigenmode_{:d}'.format(jj)+'.png')